In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier, Pool, EFeaturesSelectionAlgorithm
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from f1_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor()

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting CatBoost Preprocessor (stateless)...
Fit complete.
Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.
Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.


In [8]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
CAT_FEATURES = pre.cat_features_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']


In [13]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 66
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [14]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-12 11:00:08,614] A new study created in memory with name: no-name-b0c0ec35-726a-4053-9adc-c80394d0fac3



2. Starting Optuna study...


Best trial: 0. Best value: 0.508103:   1%|          | 1/100 [00:02<03:46,  2.28s/it]

[I 2025-11-12 11:00:10,897] Trial 0 finished with value: 0.5081033727551467 and parameters: {'learning_rate': 0.027766732918131434, 'depth': 5, 'l2_leaf_reg': 0.8660510278387105, 'subsample': 0.6, 'random_strength': 4.1368678509374466e-05, 'bagging_temperature': 0.5226685219562379, 'border_count': 210, 'scale_pos_weight': 9.96930389019623}. Best is trial 0 with value: 0.5081033727551467.


Best trial: 1. Best value: 0.559906:   2%|▏         | 2/100 [00:04<03:54,  2.40s/it]

...

[I 2025-11-12 11:03:58,318] Trial 97 finished with value: 0.574239163848755 and parameters: {'learning_rate': 0.10602857576380884, 'depth': 5, 'l2_leaf_reg': 0.0065990046889841175, 'subsample': 0.9, 'random_strength': 0.435265429698245, 'bagging_temperature': 0.6174917336524528, 'border_count': 215, 'scale_pos_weight': 3.058288806054356}. Best is trial 42 with value: 0.5856200069468566.


Best trial: 42. Best value: 0.58562:  99%|█████████▉| 99/100 [03:50<00:01,  1.38s/it]

[I 2025-11-12 11:03:59,424] Trial 98 finished with value: 0.558573165788407 and parameters: {'learning_rate': 0.08142244032562405, 'depth': 5, 'l2_leaf_reg': 0.17918476185994778, 'subsample': 0.9, 'random_strength': 0.13055094844005166, 'bagging_temperature': 0.6440027681209428, 'border_count': 208, 'scale_pos_weight': 1.5808997135958724}. Best is trial 42 with value: 0.5856200069468566.


Best trial: 42. Best value: 0.58562: 100%|██████████| 100/100 [03:52<00:00,  2.33s/it]

[I 2025-11-12 11:04:01,217] Trial 99 finished with value: 0.5231620188983637 and parameters: {'learning_rate': 0.04783917969535651, 'depth': 4, 'l2_leaf_reg': 0.7301452704902446, 'subsample': 0.9, 'random_strength': 0.7186234997773577, 'bagging_temperature': 0.4684294407820243, 'border_count': 234, 'scale_pos_weight': 7.883469830257926}. Best is trial 42 with value: 0.5856200069468566.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.5856
  Best Hyperparameters:
    learning_rate: 0.09325715313438826
    depth: 5
    l2_leaf_reg: 0.0021410243996902215
    subsample: 0.9
    random_strength: 0.04848483602713997
    bagging_temperature: 0.6979998048549101
    border_count: 239
    scale_pos_weight: 2.2902998251039337


In [23]:
best_params_from_optuna = study.best_trial.params.copy()

rfe_params = best_params_from_optuna.copy()
rfe_params.update({
    'iterations': 1000, # Max iterations per step
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 100,
    'random_state': 123,
    'verbose': False
})

train_pool = Pool(
    data=X_train_proc, 
    label=y_train, 
    cat_features=CAT_FEATURES
)

eval_pool = Pool(
    data=X_test_proc, 
    label=y_test, 
    cat_features=CAT_FEATURES
)

model_for_rfe = CatBoostClassifier(**rfe_params)

print("Starting CatBoost RFE...")
start_time = time.time()

summary = model_for_rfe.select_features(
    train_pool,
    eval_set=eval_pool,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByLossFunctionChange,
    num_features_to_select=50,
    steps=10, 
    features_for_select=X_train_proc.columns.tolist(),
    logging_level='Info', # Shows progress
    train_final_model=False
)

end_time = time.time()
print(f"RFE Complete. Took {end_time - start_time:.2f} seconds.")

selected_features = summary['selected_features_names']
eliminated_features = summary['eliminated_features_names']

print(f"\n--- RFE Results ---")
print(f"Total features selected: {len(selected_features)}")
print(f"Total features eliminated: {len(eliminated_features)}")
print("\nEliminated features:")
print(eliminated_features)

Starting CatBoost RFE...
Step #1 out of 10

liab_prct, bin=35 score 24.89008809
has_recovery_target, bin=0 score 31.65969367
highrisk_site_binary, bin=0 score 33.95795559
witness_binary, bin=0 score 35.58066597
liab_prct, bin=41 score 36.89271689

liab_prct, bin=38 score 21.52506273
multicar_binary, bin=0 score 27.33976975
highrisk_site_binary, bin=0 score 29.48784749
witness_present, bin=0 score 30.98652847
liab_prct, bin=33 score 31.67453292

liab_prct, bin=35 score 18.55382772
is_single_car, bin=0 score 23.40464238
highrisk_site_binary, bin=0 score 25.25857842
liab_prct, bin=41 score 26.66781441
witness_present_ind, value=1 score 27.87763603

liab_prct, bin=33 score 15.77310722
multicar_binary, bin=0 score 19.97392005
high_risk_site, bin=0 score 21.64194621
liab_prct, bin=41 score 23.15367943
witness_binary, bin=0 score 24.26616426

liab_prct, bin=34 score 13.60376612
recovery_case_clarity, bin=0 score 17.37590282
high_risk_site, bin=0 score 18.80911489
liab_prct, bin=44 score 20.15

In [24]:
X_train_top_features = X_train_proc[selected_features]
X_test_top_features = X_test_proc[selected_features]

original_cat_features = set(CAT_FEATURES)
top_features_set = set(selected_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

In [26]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.09325715313438826, 'depth': 5, 'l2_leaf_reg': 0.0021410243996902215, 'subsample': 0.9, 'random_strength': 0.04848483602713997, 'bagging_temperature': 0.6979998048549101, 'border_count': 239, 'scale_pos_weight': 2.2902998251039337}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5814

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.89      0.81      0.85      4165
   Class 1.0       0.51      0.68      0.58      1235

    accuracy                           0.78      5400
   macro avg       0.70      0.74      0.71      5400
weighted avg       0.81      0.78      0.79      5400



In [27]:
final_params_rfe = study.best_trial.params.copy()
final_params_rfe.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 100,
    'random_state': 123
})

new_model = CatBoostClassifier(**final_params_rfe)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

# Get the original F1 score from cell 13 to compare
original_f1_score = f1_score(y_test, best_model.predict(X_test_proc), pos_label=1)

print("\n--- Model Performance Comparison ---")
print(f"Original F1 score (all 83 features): {original_f1_score:.4f}")
print(f"New F1 score ({len(selected_features)} RFE features): {new_f1:.4f}")

print("\nNew Model Classification Report (RFE Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))


--- Model Performance Comparison ---
Original F1 score (all 83 features): 0.5814
New F1 score (50 RFE features): 0.5864

New Model Classification Report (RFE Features):
              precision    recall  f1-score   support

   Class 0.0       0.90      0.80      0.85      4165
   Class 1.0       0.51      0.69      0.59      1235

    accuracy                           0.78      5400
   macro avg       0.70      0.75      0.72      5400
weighted avg       0.81      0.78      0.79      5400



In [14]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6022.cbm")

print("Done.")

Saving best_model...
Done.


In [15]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 83

Top 10 Most Important Features:
                     feature  importance
14                 liab_prct   23.422263
49     recovery_case_clarity    6.895557
52      highrisk_site_binary    3.970960
46             is_single_car    3.845630
13       witness_present_ind    2.992513
45  is_multi_vehicle_unclear    2.951461
25       in_network_bodyshop    2.888566
53            high_risk_site    2.612414
48       has_recovery_target    2.430656
22            vehicle_weight    2.138397

Bottom 10 Least Important Features:
              feature  importance
66     veteran_driver    0.052804
43      police_binary    0.047522
62  middle_age_driver    0.035613
36          is_friday    0.028452
59       young_driver    0.023122
38         claim_hour    0.000000
64      novice_driver    0.000000
60      senior_driver    0.000000
39          rush_hour    0.000000
40         late_night    0.000000

Keeping top 66 features and removing bottom 100.


In [16]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data for CatBoost...
Applying mandatory .astype(str) to: ['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season', 'policy_report_filed_ind', 'witness_present_ind']
Dropping helper datetime columns: ['claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            0
2       4655051            0
3       6728725            1
4       9848460            1


In [17]:
prediction.to_csv("results/catboost_6022_prediction.csv", index=False)